# Práctica 1: Análisis exploratorio de datos, preprocesamiento y validación de modelos de clasificación

### Minería de Datos: Curso académico 2020-2021

### Alumnos:

* José Luis Bernáldez Morales
* Guillermo López Bermejo

Entregado el día 08/11/2020

# Introducción

En esta práctica trabajaremos los siguientes aspectos explicados en clase:

* Almacenamiento y carga de datos
* Análisis exploratorio de datos
* Preprocesamiento de datos
* Validación de modelos de clasificación

A diferencia de la libreta de ejemplo, aquí trabajaremos con las bases de datos *pima_diabetes* y *wisconsin*, las cuales cargaremos cada una en su respectivo apartado. Aunque el proceso de análisis, preprocesamiento y validación los realizaremos cada uno de acuerdo a su base de datos, tanto los imports necesarios y la semilla los estableceremos ahora, puesto que nos servirán a ambos en el desarrollo de la práctica.


Primero, incluiremos los imports

In [ ]:
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.compose import make_column_transformer, make_column_selector 
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import miner_a_de_datos_an_lisis_exploratorio_utilidad as utils

Antes de comenzar con la carga de datos, uno de los aspectos fundamentales de esta práctica es la posibilidad de reproducir los experimentos las veces que hagan falta para saber si el tratamiento de los datos es el correcto o no. Para ello, en aquellos momentos donde la aleatoriedad sea un factor que nos afecte, puesto que los experimentos deben darse en igualdad de condiciones para que nos sirva su comparatoria, vamos a establecer la semilla de aleatoriedad que vamos a usar durante toda la práctica. Su valor no es importante, pero el hecho de que usemos siempre el mismo, sí que lo es.

In [ ]:
seed = 27912 # La misma que en la libreta de ejemplo

# Pima Diabetes

# 1. Almacenamiento y carga de datos

La primera base de datos que vamos a analizar es la de Pima Diabetes, cuyo objetivo es diagnosticar si un paciente tiene diabetes basándose en una serie de mediciones diagnósticas.


Comenzamos cargando el conjunto de datos diabetes:

In [ ]:
import pandas as pd

filepath = "../input/pima-indians-diabetes-database/diabetes.csv"

target = "Outcome"

data = pd.read_csv('../input/pima-indians-diabetes-database/diabetes.csv')
data[target] = data[target].astype("category")

Obtenemos una muestra representativa aleatorizada del conjunto de datos:

In [ ]:
data.sample(5, random_state=seed)

Dividimos el conjunto en variables predictoras (X) y variable objetivo (y):

In [ ]:
(X, y) = utils.divide_dataset(data, target="Outcome")

Comprobamos que el conjunto de datos se ha dividido correctamente mediante dos muestras representativas aleatorizadas:

In [ ]:
X.sample(5, random_state=seed)

In [ ]:
y.sample(5, random_state=seed)

Dividimos el conjunto de datos en una muestra de entrenamiento (70%) y otra de pruebas (30%):



In [ ]:
train_size = 0.7

(X_train, X_test, y_train, y_test) = train_test_split(X, y,
                                                      stratify=y,
                                                      random_state=seed,
                                                      train_size=train_size)

La muestra de datos debe aleatorizarse antes de la división (shuffle=True por defecto) para evitar la eliminación de todas las instancias de una o varias clases en caso de que el conjunto de datos esté ordenado en base a los valores de la variable clase.

También debemos establecer la semilla para que los datos sean reproducibles (random_state=seed), e indicamos que el holdout sea estratificado (stratify=y) para preservar la porción de ejemplos de cada clase en cada uno de los conjuntos resultantes de la división.

Volvemos a comprobar que la división se ha realizado correctamente mediante una muestra aleatoria:

In [ ]:
X_train.sample(5, random_state=seed)

In [ ]:
X_test.sample(5, random_state=seed)

In [ ]:
y_train.sample(5, random_state=seed)

In [ ]:
y_test.sample(5, random_state=seed)

Antes de comenzar el análisis exploratorio de datos, volvemos a unir las variables predictoras con la variable clase:

In [ ]:
data_train = utils.join_dataset(X_train, y_train)
data_test = utils.join_dataset(X_test, y_test)

Comprobamos si la unión se ha realizado correctamente:

In [ ]:
data_train.sample(5, random_state=seed)

In [ ]:
data_test.sample(5, random_state=seed)

# 2. Análisis exploratorio de datos

Para obtener el número de casos y variables utilizamos el atributo shape:

In [ ]:
tamaño = data_train.shape
tamaño

Como podemos observar, tenemos 537 casos y 9 variables (8 predictoras y 1 variable clase).

Ahora para obtener el tipo de cada una de las variables utilizamos el método info:

In [ ]:
data_train.info(memory_usage=False)

Como podemos observar, del conjunto de variables predictoras, las variables *Glucose*, *BloodPressure*, *SkinThickness*, *Insulin* y *Age* son numéricas enteras, y las variables *BMI* y *DiabetesPedigreeFunction* son numéricas decimales (float).

En cuanto a la variable clase (Outcome), es una variable categórica.

In [ ]:
y_train.cat.categories

Como podemos observar, esta variable categórica Outcome únicamente contiene 2 estados, 1 y 0, que según podemos deducir de la información de la base de datos, indica si el paciente tiene o no diabetes.

### Visualización de las variables

En primer lugar mostramos el histograma que muestra la densidad de ejemplos para cada uno de los valores:

In [ ]:
utils.plot_histogram(data_train)

A primera vista podemos observar una anomalía, algunas variables como *Glucose*, *BloodPressure*, *SkinThickness*, *Insulin* y *BMI* presentan una gran cantidad de ejemplos en el primer intervalo (0-x), vamos a comprobar si existen registros con el valor 0 para estas variables y calcular la porción de la base de datos que representan:

In [ ]:
nfilas = tamaño[0]
perdidos = {}
Glucose = np.asarray(data_train[["Glucose"]])
perdidos["Glucose"] = (np.sum(Glucose == 0)/nfilas) * 100
perdidos["Glucose"]

In [ ]:
BloodPressure = np.asarray(data_train[["BloodPressure"]])
perdidos["BloodPressure"] = (np.sum(BloodPressure == 0)/nfilas) * 100
perdidos["BloodPressure"]

In [ ]:
SkinThickness = np.asarray(data_train[["SkinThickness"]])
perdidos["SkinThickness"] = (np.sum(SkinThickness == 0)/nfilas) * 100
perdidos["SkinThickness"]

In [ ]:
Insulin = np.asarray(data_train[["Insulin"]])
perdidos["Insulin"] = (np.sum(Insulin == 0)/nfilas) * 100
perdidos["Insulin"]

In [ ]:
BMI = np.asarray(data_train[["BMI"]])
perdidos["BMI"] = (np.sum(BMI == 0)/nfilas) * 100
perdidos["BMI"]

Como podemos observar, efectivamente tenemos registros con el valor 0 para variables en las que no tendría sentido (es imposible tener una presión sanguínea o un nivel de insulina de 0), por lo que los consideraremos valores perdidos.

Vamos a visualizar los valores perdidos de cada atributo de manera gráfica:

In [ ]:
df = pd.DataFrame(data=perdidos, index = ['Perdidos'])
df = df.transpose()
df.columns = ['Perdidos']
df.index.names = ['Variables']
df['Variables'] = df.index

In [ ]:
utils.CustomBarplot(df, 'Variables', 'Perdidos')

Podemos destacar el 48,6% de valores perdidos de la variable *Insulin* (prácticamente la mitad de los registros), y el 29% de valores perdidos de la variable *SkinThickness*.

A primera vista esto hace que estas dos variables sean susceptibles de ser eliminadas.

Ahora vamos a mostrar una tabla resumen con información relevante como la media, la mediana, la desviación estándar, etc. de cada una de las variables.

In [ ]:
data_train.describe(include="number")

A primera vista podemos observar una gran desviación estándar en la variable *Insulin*, entre otras variables, lo que podría indicar una gran cantidad de outliers en estas variables.

Vamos a tratar de visualizar estos posibles outliers con un diagrama de caja y bigotes:

In [ ]:
utils.BoxWhisker(data_train, (3,3))

Como podemos observar, las variables *Insulin* y *DiabetesPedigreeFunction* presentan un número considerable de outliers, por lo que son susceptibles de ser eliminadas.

En cuanto al resto de variables, la mayoría también presentan outliers, por lo que a la hora de imputar los valores perdidos, debemos sustituirlos por la mediana y no por la media, ya que esta se ve mucho más afectada por los outliers.

Ahora vamos a visualizar las variables categóricas del problema:

In [ ]:
utils.plot_barplot(data_train)

Como podemos observar hay aproximadamente un 65,17% de casos con el valor 0, y un 34,82% de casos con el valor 1, esto nos da a entender que el conjunto de datos no está balanceado y por tanto, dado que hemos indicado que se trata de un holdout estratificado al realizar las particiones, esta proporción deberá mantenerse prácticamente igual en el conjunto de datos de test:

In [ ]:
utils.plot_barplot(data_test)

Como podemos observar, hay aproximadamente una frecuencia del 64,93% de casos para el valor 0, y del 35,06% para el valor 1, valores casi idénticos a los obtenidos para la distribución de entrenamiento.

Este análisis univariado nos permite identificar problemas como ruido y outliers, así como distribuciones carentes de información. Para obtener información algo más relevante, debemos realizar un análisis multivariado que contraste el conjunto de variables para determinar la potencia discriminativa de los atributos en base a la información que aportan sobre la variable clase.

En primer lugar vamos a estudiar las relaciones entre pares de variables mediante gráficos de nube de puntos organizados en una matriz en los que cada punto corresponde a un caso coloreado según la variable clase a la que pertenezca y en cada eje se representa un atributo.

In [ ]:
utils.plot_pairplot(data_train, target="Outcome")

A primera vista no se observa ninguna variable con un poder discriminativo destacable.

Ahora vamos a visualizar las posibles correlaciones entre variables mediante un mapa de calor:

In [ ]:
utils.HeatMap(X_train)

No se observa ninguna correlación que aporte información relevante de cara al preprocesamiento, aunque resulta interesante observar que si que existe una mínima correlación con un valor de 0.56 sobre 1 entre las variables *Age* y *Pregnancies*, ya que a más edad, más probabilidad de haber tenido hijos.

# 3. Preprocesamiento de datos

### Imputación de valores perdidos y selección de variables

Para llevar a cabo la imputación vamos a sustituir los valores perdidos de las variables *Glucose*, *BloodPressure*, *SkinThickness*, *BMI* y *DiabetesPedigreeFunction* por la mediana de cada uno de los atributos, ya que, como comentamos en el análisis exploratorio, es el parámetro más adecuado para variables con outliers.

Para ello crearemos un imputador utilizando la función SimpleImputer de sklearn para facilitarnos su inclusión en el pipeline:

In [ ]:
impute = make_pipeline(SimpleImputer(strategy="median", missing_values=0))

columns = 'Glucose|BloodPressure|SkinThickness|BMI|DiabetesPedigreeFunction'
non = 'Pregnancies'

imputer = make_column_transformer(
    (impute, make_column_selector(pattern=columns)),
    ('passthrough', make_column_selector(pattern=non))
)

En cuanto a la selección de variables, se ha optado por eliminar la variable *Insulin* por tres motivos principales:

* La gran cantidad de valores perdidos (casi la mitad de los registros para esta variable son valores perdidos, siendo esta la que más tiene).
* El elevado número de outliers.
* La ausencia de poder predictivo observada en el análisis exploratorio.

Esta eliminación de la variable *Insulin* se ha realizado de manera implícita al no incluirla en la lista columns a la hora de crear el imputador.

Esta sería una muestra del array obtenido tras aplicar la imputación. Como se puede observar, la columna correspondiente a los registros de la variable Insulin ha sido eliminada, y los valores perdidos (registros con un 0) han sido sustituidos por la mediana de la variable.

In [ ]:
imputer.fit_transform(X_train)

En cuanto a la discretización, dado que tras el análisis exploratorio no hemos obtenido ninguna información relevante que nos induzca a pensar que una discretización por igual anchura o frecuencia sea lo mejor, vamos a optar por utilizar un método de discretización no supervisada algo más complejo, llamado discretización por k-medias con dos secciones.

In [ ]:
discretizer = KBinsDiscretizer(n_bins=2, strategy="kmeans")

# 4. Algoritmos de clasificación

### Algoritmo Zero-R

Este es el algoritmo que utilizaremos como baseline para evaluar la complejidad del conjunto de datos y la efectividad de los clasificadores, considerando éste como el peor clasificador posible.

In [ ]:
zero_r_model = DummyClassifier(strategy="most_frequent")

### Algoritmo CART
Ahora probamos con un algoritmo algo más elaborado tal como un árbol de decisión:

In [ ]:
tree_model = DecisionTreeClassifier(random_state=seed)

### Pipeline
Definimos el pipeline con la secuencia de preprocesamiento que queremos aplicar antes del aprendizaje del árbol de decisión mediante make_pipeline, compuesto por el imputador/selector de variables y el discretizador:

In [ ]:
pipeline = make_pipeline(imputer,discretizer,tree_model)

# 5. Evaluación de modelos

Ahora pasamos a entrenar y validar los modelos mediante matrices de confusión, la tasa de acierto y otras métricas.

Comenzamos con el algoritmo Zero-R:

### Evaluar algoritmo *Zero-R*

In [ ]:
utils.evaluate(zero_r_model,
               X_train, X_test,
               y_train, y_test)

In [ ]:
utils.metrics(zero_r_model,
               X_train, X_test,
               y_train, y_test)

Como podemos observar, obtenemos un accuracy de 0,64935, que es precisamente el porcentaje de casos de la clase mayoritaria (valor 0) en la base de datos, por lo que a pesar de que está por encima del 50% de accuracy podemos considerarlo como un mal resultado.

Esto se confirma al observar que, como era de esperar, los valores tanto de precision como de recall para el valor 1 son de 0.

### Evaluar algoritmo *CART*

In [ ]:
utils.evaluate(tree_model,
               X_train, X_test,
               y_train, y_test)

Como podemos observar, hemos obtenido un accuracy ligeramente superior al obtenido con el algoritmo Zero-R, pero seguimos sin poder considerarlo como un buen resultado.

De cara a su comparación con el algoritmo aplicando el pipeline vamos a mostrar otras métricas como el recall o la precisión.

In [ ]:
utils.metrics(tree_model,
               X_train, X_test,
               y_train, y_test)

Por último vamos a evaluar el árbol de decisión obtenido aplicando un preprocesamiento que incluiría una selección de variables, la imputación de valores y la discretización por k-medias:

### Evaluar clasificador con el Pipeline

In [ ]:
utils.evaluate(pipeline,
               X_train, X_test,
               y_train, y_test)

In [ ]:
utils.metrics(pipeline,
               X_train, X_test,
               y_train, y_test)

Podemos observar un pequeño incremento del accuracy o tasa de acierto (de 0,65 a 0,69).

En cuanto a otras métricas, podemos observar una disminución del recall, esto implica una reducción de la fracción de casos positivos detectados.

# 5. Conclusiones

En conclusión, tras aplicar el preprocesamiento formado por una selección de variables, una imputación de valores y una discretización por k-medias hemos obtenido un mejor resultado en cuanto a la tasa de acierto a cambio de una reducción del recall, por lo que este preprocesamiento podría no ser adecuado a la hora de utilizar este modelo para el diagnóstico de la diabetes, ya que habrá un mayor número de casos positivos que no serán detectados como tal.

# Breast Cancer Wisconsin

# 1. Almacenamiento y carga de datos

La segunda base de datos con la que vamos a trabajar es *Breast Cancer Wisconsin data*, que es un dataset de 569 muestras en el que se trata de clasificar si un cáncer de mama es benigno o maligno. Para ello, y como ya hemos puesto en la carga de datos, la variable objetivo de la que nos encargaremos de clasificar será el diagnóstico, variable discreta cuyos valores serán **Benigno**/**Maligno**, y para identificar cada una de las instancias, utilizaremos una variable de tipo entero que servirá de identificador.

En cuanto a las variables predictoras, en nuestro caso, tendremos 10:
* Radius: media de las distancias del centro al perímetro
* Texture: desviación estandar de valores en escala de grises
* Perimeter
* Area
* Smoothness: variación local en la longitud de los radio
* Compactness: perímetro^2 / area - 1
* Concavity: severidad de las porciones cóncavas del contorno
* Symmetry
* Fractal dimension: "coastline approximation" - 1


Ahora, cargaremos las bases de datos e imprimimos 5 instancias aleatorias para ver que está todo correcto.

In [ ]:
filepath = "../input/breast-cancer-wisconsin-data/data.csv"

index = "id"
target = "diagnosis"   # Nuestra variable objetivo

data = utils.load_data(filepath, index, target)
data.sample(5, random_state=seed)

Antes de nada, y analizando esta pequeña tabla, podemos ver que la última variable de todas, *Unnamed*, muestra valores null, es decir, que no aporta valor para nuestro problema. Para ver si tenemos que eliminar estos valores, vamos a comprobar que no estén vacíos todos los registros:

In [ ]:
data.info(memory_usage=False)

De esta manera, podemos ver que esa variable no nos aporta información, por lo que vamos a eliminar estos valores. Además, la variable id tampoco nos va a servir para la clasificación, porque no es una variable predictora.

Esta tarea es parte del preprocesamiento de datos, pero no tiene sentido seguir arrastrando este error, por lo que hemos decidido quitar la variable de en medio.

In [ ]:
data = data.drop('Unnamed: 32', axis=1)

Dividamos ahora el nuestro dataset:

In [ ]:
(X, y) = utils.divide_dataset(data, target="diagnosis")

X.sample(5, random_state=seed)

In [ ]:
y.sample(5, random_state=seed)

Ahora que ya tenemos los datos, tenemos que asegurarnos de que el trabajo que vamos a realizar lo hacemos de manera correcta. Para ello, vamos a aplicar un holdout, el cual consiste en dividir nuestra base de datos en dos muestras diferentes, una para el entrenamiento del modelo y otra para la validación de los datos (la distribución será 70% - 30%).

El primer conjunto lo utilizaremos para entrenar y así crear la estimación del modelo, pero no podremos usar el 30% restante para entrenar, sino que únicamente los utilizaremos una vez para comprobar que el model funciona como esperamos. De esta manera, evitamos el sobreajuste, puesto que si los datos de validación se utilizaran para aprender el modelo, no estaríamos probando nuestro modelo en datos nuevos, y la estimación de acierto sería optimista e incorrecta. Es decir, estaríamos sobreajustando el modelo a los datos que vamos a usar para validarlo, y lo que queremos es un clasificador que generalice y nos sirva para datos nuevos.

Es muy importante que en el proceso de creación de dichas carpetas, aleatoricemos sus registros de manera que la muestra no esté sesgada. Para ello, la siguiente función tiene por defecto un parámetro, llamado shuffle, que se encuentra a True por defecto, por lo que no tendremos que hacerlo a mano. Del mismo modo, aún no sabemos cómo está distribuida la muestra que nos han dado, es decir, no sabemos si está o no balanceada, por lo que vamos a estratificar, es decir, preservar la distribución original de la variable clase en cada uno de los conjuntos que vamos a crear.

In [ ]:
train_size = 0.7

(X_train, X_test, y_train, y_test) = train_test_split(X, y,
                                                      stratify=y,
                                                      random_state=seed,
                                                      train_size=train_size)
X_train.sample(5, random_state=seed)
# X_test.sample(5, random_state=seed)

In [ ]:
y_train.sample(5, random_state=seed)
# y_test.sample(5, random_state=seed)

Estas variables serán las que utilicemos para entrenar nuestros modelos, pero para la visualización de los datos, vamos a volver a unirlos, ya que nos serán más útiles que por separado. Importante no unir el conjunto de datos con el conjunto de test para evitar una fuga de datos.

In [ ]:
data_train = utils.join_dataset(X_train, y_train)
# data_train.sample(5, random_state=seed)

Puesto que las variables predictoras son 10, pero están divididas en valor medio, error y peor valor, vamos a separarlos para que más adelante podamos comparar dichas variables con mayor facilidad.

In [ ]:
# Creamos listas para eliminar las columnas que necesitemos
variables_mean = ['radius_mean', 'texture_mean','perimeter_mean','area_mean','smoothness_mean','compactness_mean','concavity_mean','concave points_mean','symmetry_mean','fractal_dimension_mean']
variables_se = ['radius_se', 'texture_se','perimeter_se','area_se','smoothness_se','compactness_se','concavity_se','concave points_se','symmetry_se','fractal_dimension_se']
variables_worse = ['radius_worst', 'texture_worst','perimeter_worst','area_worst','smoothness_worst','compactness_worst','concavity_worst','concave points_worst','symmetry_worst','fractal_dimension_worst']
variables = variables_mean + variables_se + variables_worse

meanList = ['radius_se', 'texture_se','perimeter_se','area_se','smoothness_se','compactness_se','concavity_se','concave points_se','symmetry_se','fractal_dimension_se','radius_worst', 'texture_worst','perimeter_worst','area_worst','smoothness_worst','compactness_worst','concavity_worst','concave points_worst','symmetry_worst','fractal_dimension_worst']
seList = ['radius_mean', 'texture_mean','perimeter_mean','area_mean','smoothness_mean','compactness_mean','concavity_mean','concave points_mean','symmetry_mean','fractal_dimension_mean','radius_worst', 'texture_worst','perimeter_worst','area_worst','smoothness_worst','compactness_worst','concavity_worst','concave points_worst','symmetry_worst','fractal_dimension_worst']
worstList = ['radius_mean', 'texture_mean','perimeter_mean','area_mean','smoothness_mean','compactness_mean','concavity_mean','concave points_mean','symmetry_mean','fractal_dimension_mean','radius_se', 'texture_se','perimeter_se','area_se','smoothness_se','compactness_se','concavity_se','concave points_se','symmetry_se','fractal_dimension_se']

mean = X_train.drop(meanList, axis=1)
se = X_train.drop(seList, axis=1)
worst = X_train.drop(worstList, axis=1)

# 2. Análisis exploratorio de los datos

### Descripción del conjunto de datos

Esta es la parte de la práctica donde comenzaremos a analizar las variables de manera que podamos obtener información sobre ellas. Vamos a empezar analizando el tamaño del problema:

In [ ]:
data.shape

Podemos ver que el tamaño de la muestra, como ya sabíamos, es de 569 instancias, las cuales tienen 30 varaibles cada una, además de contar con la variable clase y el id. 

In [ ]:
data_train.info(memory_usage=False)

Nuestro conjunto de entrenamiento se compone de 398 instancias, con 30 variables predictoras cada una. De todas estas variables, podemos ver que son todas numéricas, concretamente decimales, a excepción de la variable clase, que es categórica. Ahora vamos a comprobar si tenemos valores perdidos.

In [ ]:
missing_values = data_train.isnull().sum()
missing_values

Veamos la variable clase:

In [ ]:
y_train.cat.categories

Sus valores son benigno (B) y maligno (M). Veamos ahora su distribución.

### Visualización de las variables

In [ ]:
utils.plot_barplot(data_train)

Aquí podemos ver que la muestra no está balanceada, ya que hay más casos benignos que malignos. De aquí la importancia de que hayamos realizado un holdout estratificado, para que la proporción de casos benignos y malignos se mantenga y no perdamos información.

Realicemos ahora un análisis univariado, en el que podamos ver como se distribuyen los valores de cada una de las variables predictoras. Utilizaremos histogramas para visualizar los datos y obtendremos también las tablas para ver los valores. Puesto que tenemos 30 variables predictoras, resultará más interesante realizar varios histogramas, para que podamos compara los valores de las variables que corresponden al mismo tipo de dato (media, error, peor caso):

In [ ]:
utils.plot_histogram(mean)

In [ ]:
mean.describe()

Prácticamente todas tienen una distribución normal o casi normal. Otras, como *concavity_mean* o *concave points_mean* son exponenciales.

In [ ]:
utils.plot_histogram(se)

In [ ]:
se.describe()

En el caso del error, la tendencia cambia, puesto que podemos ver que algunas de las variables, como *radius_se* y *perimeter_se* pasan de ser normales a exponenciales, mientras que *concave points_se* es normal en vez de exponencial.

In [ ]:
utils.plot_histogram(worst)

In [ ]:
worst.describe()

Por último, los valores en los peores casos son, en su mayoría, distribuciones normales o mixturas. De estas gráficas podemos observar algo interesante, y es que la mayoría de variables parecen tener outliers, por lo que conocer sus distribuciones nos ayudará a imputar de la mejor manera dichos valores si fuera necesario. Para comprobarlo, utilizaremos diagramas de cajas:

In [ ]:
utils.BoxWhisker(data_train, (6,6))

Efectivamente podemos comprobar que nuestras variables tienen outliers o ruido, por lo que consideraremos estos valores como erróneos.

También resulta interesante comprobar si las variables están correlacionadas entre sí. Para ello, crearemos una matriz de gráficos, y en vez de crear 1 de los datos en general, utilizaremos las particiones donde tenemos las variables predictoras agrupadas para realizar el análisis multivariado.

In [ ]:
# Introducimos el valor de la variable clase en nuestras tablas para poder visualizar los datos mejor
mean = data_train.drop(meanList, axis=1)
se = data_train.drop(seList, axis=1)
worst = data_train.drop(worstList, axis=1)

In [ ]:
utils.plot_pairplot(mean, target='diagnosis')

In [ ]:
utils.plot_pairplot(se, target='diagnosis')

In [ ]:
utils.plot_pairplot(worst, target='diagnosis')

 El problema de tener tantas variables, es que no podemos sacar conclusiones de manera sencilla, puesto que las gráficas no son muy grandes y es probable que cometamos errores. No obstante, no está mal tenerlas para al menos descartar algunas teorías y quizá formular otras. Por ejemplo, a simple vista, se puede ver que la variable *radius_mean* muy probablemente estará correlacionada con las variables *perimeter_mean* y *area_mean*, aunque esto no debería sorprendernos porque son elementos que ya están relacionados en otros ámbitos matemáticos. Esto se aplica también en los valores *se* y *worst*.

Otras variables, como el *fractal_dimension* o el *texture_mean* ya las podemos ir descartando porque no parece que vayan a ofrecernos una gran cantidad de información relevante para nuestro problema. También podemos observar que las variables de las medias son valores más razonables a la hora de utilizarlas en un posible clasificador, mientras que con el error y el peor caso, cometeríamos un error mayor y presentan un mayor número de outliers, por lo que si no fueramos a ocuparnos de ellos, los podríamos descartar.
 
Para ver mejor las posibles correlaciones, vamos a analizar los mapas de calor.

In [ ]:
utils.HeatMap(mean)

In [ ]:
mean.corr()

Efectivamente, el radio está correlacionado con el perímetro y el área. Además, podemos ver que la variable *concave points_mean* también tiene una fuerte correlación con estas 3 variables, así como con *concavity mean* y *compactness_mean*.

In [ ]:
utils.HeatMap(se)

In [ ]:
se.corr()

In [ ]:
utils.HeatMap(worst)

In [ ]:
worst.corr()

De los otros dos mapas de calor, a parte de las relaciones ya establecidas, podemos ver que tampoco encontramos correlaciones negativas en ninguna de las variables, que las variables que representan valores medios parecen más interesantes y que hay 2 variables más que nos pueden ser útiles, que son *concavity* y *concave_points*, tanto en sus valores medios como en el peor.

Por tanto, de los mapas de calor y las gráficas previamente creadas, las variables que a priori parecen importantes y que afectan a la distribución son el radio, el perímetro y el área. Otras variables como la concavidad nos pueden ser útiles también. Es importante conocer estos valores de manera que podamos evitar utilizar más variables de las necesarias, facilitándonos el proceso.

Ahora, vamos a utilizar dichas variables para ver como afectan a la variable clase y así, probablemente, reducir el número de variables que utilizará nuestro predictor.

In [ ]:
for i in variables:
    fig = px.histogram(X_train, x=i, color=y_train, )
    fig.update_traces(opacity=0.7)
    fig.update_layout(barmode='overlay') # Así vemos como se superponen
    fig.show()

De aquí podemos observar qué variables, como *radius_mean*, *perimeter_mean*, etc., discriminan mejor la variable clase.

# 3. Preprocesamiento de datos

En este apartado vamos a trabajar sobre las operaciones que vamos a realizar sobre nuestros datos crudos. Estas operaciones van a ser las siguientes:
* Limpieza de datos: eliminaríamos la variable *Unnamed: 32*, puesto que no aporta ningún valor, aunque ya lo hemos hecho.
* Reducción de datos: seleccionaremos qué variables vamos a utilizar para discriminar la variable clase y realizaremos una discretización de los mismos.
* Por último, cambiaremos los valores de la variable objetivo por ceros y unos de manera que podamos comparar los modelos utilizando la gráfica ROC.

Empecemos con la selección de variables.

Para comenzar, puesto que las variables *radius*, *perimeter* y *area* están claramente correlacionadas, podemos eliminar 2 de ellas. En nuestro caso, nos quedaremos con el radio.

Las variables *concavity* y *concave points* también lo están, por lo que nos quedaremos con la concavidad.

El resto de variables las vamos a omitir.

In [ ]:
# Estas son todas las variables a eliminar
variables_mean_drop = ['perimeter_mean','area_mean','concave points_mean']
variables_se_drop = ['perimeter_se','area_se','concave points_se']
variables_worse_drop = ['perimeter_worst','area_worst','concave points_worst']

reduccion_datos = variables_mean_drop + variables_se_drop + variables_worse_drop

In [ ]:
elim_var = make_column_transformer(("drop", reduccion_datos), remainder="passthrough")

Ahora transformaremos la variable clase en numérica.

In [ ]:
data['diagnosis']=data['diagnosis'].map({'M':1,'B':0})
(X, y) = utils.divide_dataset(data, target="diagnosis")
(X_train, X_test, y_train, y_test) = train_test_split(X, y,
                                                      stratify=y,
                                                      random_state=seed,
                                                      train_size=train_size)

# Como estamos utilizando la misma semilla, la división se realizará de la misma manera, por lo que 
# los datos visualizados y estos serán exactamente iguales

Por último, discretizaremos las variables mediante k-medias, utilizando 4 secciones, puesto que no podemos dividir exactamente las variables ya que tendríamos resultados incorrectos.

In [ ]:
discretizer = KBinsDiscretizer(n_bins=4, strategy="kmeans")

Para poder aplicar todas estas transformaciones, haremos unos de un *pipeline*, el cual aplicará automáticamente las transformaciones implementadas en este apartado a cualquier conjunto de datos que le pasemos.

# 4. Algoritmos de clasificación y evaluación de modelos

Vamos a implementar ahora los algoritmos y procederemos con el modelado. Una vez se hayan entrenado los algoritmos, procederemos a validar y evaluar los modelos, y escogeremos el mejor.

Para implementar el *Zero-R*, es necesario que definamos el hiperparámetro de la estrategia como *most_frequent*, de manera que el clasificador clasifique todas las instancias como la clase mayoritaria. 

En cuanto al árbol de decisión, será necesario indicar la semilla que estamos utilizando en el hiperparámetro del *random_state*, puesto que en caso de empate entre diferentes variables, siempre elija las mismas con el objetivo de que los resultados de esta práctica sean reproducibles.

In [ ]:
zero_r_model = DummyClassifier(strategy="most_frequent")
tree_model = DecisionTreeClassifier(random_state=seed)

In [ ]:
pipeline = make_pipeline(elim_var, discretizer, tree_model)

### Evaluar algoritmo *Zero-R*

In [ ]:
utils.evaluate(zero_r_model,
               X_train, X_test,
               y_train, y_test)

In [ ]:
utils.metrics(zero_r_model,
               X_train, X_test,
               y_train, y_test)

El resultado es el esperable, puesto que al utilizar la clase mayoritaria, que en este caso es la clase benigno, ha clasificado los 64 de la otra clase de manera incorrecta. Este clasificador no nos sirve para nada más que como un baseline, es decir, un punto de partida desde el cual los clasificadores que creemos nosotros tienen que mejorar su resultado.

### Evaluar algoritmo CART

In [ ]:
utils.evaluate(tree_model,
               X_train, X_test,
               y_train, y_test)

In [ ]:
utils.metrics(tree_model,
               X_train, X_test,
               y_train, y_test)

El resultado es lógicamente superior, y podemos ver lo sencillo que es realizar un clasificador relativamente bueno para un problema tan complejo. No obstante, aún podemos mejorarlo más si le aplicamos el pipeline que hemos definido previamente.

### Evaluar clasificador con el Pipeline

In [ ]:
pipeline = make_pipeline(elim_var, discretizer, tree_model)

utils.evaluate(pipeline,
               X_train, X_test,
               y_train, y_test)

In [ ]:
utils.metrics(pipeline,
               X_train, X_test,
               y_train, y_test)

Una vez aplicado el pipeline, nuestras clasificaciones son mucho más precisas. Realicemos ahora una evaluaciónde los modelos.

Para elegir el mejor modelo, nos vamos a apoyar en el análisis ROC (*Reciever Operating Characteristics*), en el cual normalizamos la matriz de confusión por columnas. Tal y como vamos a observar en los gráficos, el peor clasificador posible, en este caso el *Zero-R*, se verá representado como una recta ascendente (la representaremos como una línea discontinua). Cuanto más esté nuestra gráfica por encima del baseline, mejor será nuestro modelo.

In [ ]:
utils.ROC(tree_model,
               X_train, X_test,
               y_train, y_test)

In [ ]:
utils.ROC(pipeline,
               X_train, X_test,
               y_train, y_test)

Podemos ver que gracias al uso del pipeline hemos maximizado el área bajo la curva. Además, como la subida es tán rápida, podemos concluir de la propia gráfica lo que ya habíamos visto en los datos, y es que nuestro clasificador tiene una sensibilidad alta (tasa de verdaderos positivos) y una especifidad baja (tasa de falsos negativos).

# 5. Conclusiones

Gracias al estudio de este dataset hemos comprendido la importancia del preprocesamiento de datos y del uso del pipeline. Utilizando un árbol de decisión hemos obtenido un clasificador bastante bueno sin necesidad de trabajar sobre los datos, pero si analizamos el problema, nuestro cometido es clasificar los máximos casos malignos que podamos, pues si nuestro clasificador los pasa por alto, estaríamos poniendo vidas en juego. 

Mediante las transformaciones que hemos definido en nuestro preprocesamiento de datos hemos tratado de aumentar el recall al máximo posible para que se nos escapen los mínimos casos malignos posibles, es decir, buscamos clasificar correctamente para evitar el mayor número de falsos negativos, por lo que, en definitiva, gracias al correcto tratamiento de los datos hemos aumentado la eficacia de nuestro clasificador para el cometido que se esperaba.